# Model Playground

In [1]:
import numpy as np
import heapq

## Iteration 1

In [2]:
class MenuItem:
    def __init__(self, menu_item_id, title, category, description, image_url, price, restaurant_id,
                 carbs, protein, fat, calories, ingredients):
        self.menu_item_id = menu_item_id
        self.title = title
        self.category = category
        self.description = description
        self.image_url = image_url
        self.price = price
        self.restaurant_id = restaurant_id
        self.carbs = carbs
        self.protein = protein
        self.fat = fat
        self.calories = calories
        self.ingredients = ingredients

    @classmethod
    def construct_from_dict(cls, menu_item_dict):
        return MenuItem(menu_item_dict.menu_item_id, menu_item_dict.title, menu_item_dict.category,
                        menu_item_dict.description, menu_item_dict.image_url, menu_item_dict.price,
                        menu_item_dict.restaurant_id, menu_item_dict.carbs, menu_item_dict.protein,
                        menu_item_dict.fat, menu_item_dict.calories, menu_item_dict.ingredients)

In [5]:
def cosine_similarity(x, y):
    norm_x = np.linalg.norm(x)
    norm_y = np.linalg.norm(y)
    return x.dot(y) / (norm_x * norm_y)

def avg_cos_sim_affinity(menu_item_vec, historical_orders_vecs):
    total_cosine_sim = 0
    for historical_order_vec in historical_orders_vecs:
        total_cosine_sim += cosine_similarity(menu_item_vec, historical_order_vec)
    return total_cosine_sim / len(historical_orders_vecs)

def construct_feature_vector(menu_item):
    feature_vec = []
    feature_vec.append(menu_item.price)
    feature_vec.append(menu_item.carbs)
    feature_vec.append(menu_item.protein)
    feature_vec.append(menu_item.fat)
    feature_vec.append(menu_item.calories)
    return np.array(feature_vec)

def construct_feature_vectors(items):
    return [construct_feature_vector(item) for item in items]

class HeapNode:
    def __init__(self, index, value):
        self.index = index
        self.value = value
    def __eq__(self, other):
        return self.value == other.value
    def __lt__(self, other):
        return self.value < other.value
    
def select_top_suggestions(historical_orders, menu_items, affinity_measure, num_elem):
    pq = heapify([])
    for index in range(len(menu_items)):
        heappush(pq, HeapNode(index, menu_items[index]))
    return [heappop(pq).value for _ in range(num_elem)]

def normalize_demean(vecs):
    for vec in vecs:
        for index in range(len(vec)):
            vec[index] = vec[index] - global_mean[index]
            vec[index] = vec[index] / sqrt(global_variance[index])

In [6]:
def run_suggestion_model(historic_orders, menu_items):
    num_to_return = min(len(menu_items), 3)
    historical_orders_vecs = construct_feature_vectors(historic_orders)
    menu_item_vecs = construct_feature_vectors(menu_items)
    normalize_demean(historic_orders)
    normalize_demean(menu_item_vecs)
    return select_top_suggestions(historical_orders_vec, menu_items_vec, avg_cos_sim_affinity, num_to_return)